## This section starts us off (api query and processing) 
* This section loops through 5 dates and gets the hrly temp, humidity and wind_speed for 5 days

In [1]:
'''
The new goal of my project is to: calculate the average temp in manchester over the last 5 days and graph it
'''
# unix time calculator -0> https://www.vultr.com/resources/unix-time-calculator/?convert=date&date_day=27&date_month=10&date_year=2021&date_hour=12&date_minute=00&date_second=00&date_tz_from=UTC&date_tz_to=UTC&date_format=Y-m-d+H%3Ai%3As

import requests, json, datetime, pandas as pd

df_final = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe
last_5_days = []

for x in range(0,5): # gets current date and time delta in unixtime
    unix_date_time = (datetime.datetime.today() - datetime.timedelta(x)).strftime("%s") 
    last_5_days.append(unix_date_time)




### Manchester dataframe processing

* this gets all the data for the city of manchester, condences it to what we care about and puts it in a dataframe

In [2]:
df_final_manchester = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe

for day in last_5_days: # gets the json from the api, processes it into a df and appends it to the final frame. ownly takes 4 columbs
    response_manchester = requests.get("http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=42.9956&lon=-71.4548&dt="+ day +"&appid=fd3ef9b217de69f13fde1bd0325da289")
    
    processed_responce_manchester = response_manchester.json() # gets json
    
    df_raw_manchester = pd.DataFrame(processed_responce_manchester['hourly']) # json -> df from the "hourly" list
    
    df_processed_manchester = df_raw_manchester[["dt","temp", "humidity", "wind_speed"]].copy() # cuts df down to what we want ownly
    
    df_final_manchester = df_final_manchester.append(df_processed_manchester) # appends to the final df

df_final_manchester.insert(loc=len(df_final_manchester.columns), column='city', value='Manchester')

df_final_manchester['id'] = df_final_manchester[['dt', 'temp', 'humidity', 'wind_speed',]].sum(axis=1).map(hash)

df_final_manchester#.shape 

,dt,temp,humidity,wind_speed,city,id
0,1637884800,278.14,68,1.47,Manchester,1406564545121494235
1,1637888400,277.96,71,1.96,Manchester,2121375746036347630
2,1637892000,277.50,73,1.80,Manchester,691752794450837760
3,1637895600,277.56,71,1.82,Manchester,876220059266076430
4,1637899200,278.09,68,0.00,Manchester,207525674555039002
...,...,...,...,...,...,...
19,1637607600,282.73,68,2.24,Manchester,2236667786545588752
20,1637611200,281.74,66,0.89,Manchester,1452681361325029404
21,1637614800,280.23,65,1.79,Manchester,46116817841423915
22,1637618400,279.10,62,2.68,Manchester,1798557482853602359


### Concord data processing

* this gets all the data for the city of concord, condences it to what we care about and puts it in a dataframe

In [3]:
df_final_concord = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe

for day in last_5_days: # gets the json from the api, processes it into a df and appends it to the final frame. ownly takes 4 columbs
    response_concord = requests.get("http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=43.2081&lon=-71.5376&dt="+ day +"&appid=fd3ef9b217de69f13fde1bd0325da289")
    
    processed_responce_concord = response_concord.json() # gets json
    
    df_raw_concord = pd.DataFrame(processed_responce_concord['hourly']) # json -> df from the "hourly" list
    
    df_processed_concord = df_raw_concord[["dt","temp", "humidity", "wind_speed"]].copy() # cuts df down to what we want ownly
    
    df_final_concord = df_final_concord.append(df_processed_concord) # appends to the final df

df_final_concord.insert(loc=len(df_final_concord.columns), column='city', value='Concord')

df_final_concord['id'] = df_final_concord[['dt', 'temp', 'humidity', 'wind_speed',]].sum(axis=1).map(hash)

df_final_concord #.shape 

,dt,temp,humidity,wind_speed,city,id
0,1637884800,278.36,64,0.00,Concord,830103243062256854
1,1637888400,276.31,73,0.00,Concord,714811202552738541
2,1637892000,276.03,74,0.00,Concord,69175225943605502
3,1637895600,275.79,76,0.00,Concord,1821615890955783951
4,1637899200,276.53,77,0.00,Concord,1222096730550459681
...,...,...,...,...,...,...
19,1637607600,282.32,68,3.60,Concord,2121375196280252945
20,1637611200,281.32,65,6.17,Concord,1129863098142554144
21,1637614800,279.98,63,5.66,Concord,1475739769426937388
22,1637618400,279.04,65,6.17,Concord,484227121533424702


## Keen data processing
* this gets all the data for the city of concord, condences it to what we care about and puts it in a dataframe

In [4]:
df_final_keen = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe

for day in last_5_days: # gets the json from the api, processes it into a df and appends it to the final frame. ownly takes 4 columbs
    response_keen = requests.get("http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=41.5929&lon=-75.3738&dt="+ day +"&appid=fd3ef9b217de69f13fde1bd0325da289")
    
    processed_responce_keen = response_keen.json() # gets json
    
    df_raw_keen = pd.DataFrame(processed_responce_keen['hourly']) # json -> df from the "hourly" list
    
    df_processed_keen = df_raw_keen[["dt","temp", "humidity", "wind_speed"]].copy() # cuts df down to what we want ownly
    
    df_final_keen = df_final_keen.append(df_processed_keen) # appends to the final df

df_final_keen.insert(loc=len(df_final_keen.columns), column='city', value='keen')

#df_final_keen['id'].apply(lambda x : df_final_keen[x] = hash)

#df['a'] = df['a'].apply(lambda x: x + 1)

df_final_keen['id'] = df_final_keen[['dt', 'temp', 'humidity', 'wind_speed',]].sum(axis=1).map(hash)



df_final_keen #.shape

,dt,temp,humidity,wind_speed,city,id
0,1637884800,276.41,91,0.95,keen,830103792818070768
1,1637888400,276.28,92,0.67,keen,2190550970342060800
2,1637892000,276.29,93,1.03,keen,737869610654646546
3,1637895600,276.33,93,1.17,keen,1152921506244742946
4,1637899200,276.02,94,1.32,keen,783986426858462515
...,...,...,...,...,...,...
19,1637607600,276.41,60,4.55,keen,2213609378443684356
20,1637611200,275.44,54,4.21,keen,1498798177528838157
21,1637614800,274.30,61,4.38,keen,1567973401834554915
22,1637618400,273.24,67,3.32,keen,1291271954855892023


### This section merged the 3 newly ceated data frames

In [5]:
df_big = pd.concat([df_final_manchester, df_final_keen, df_final_concord], axis=0)

## this sends df_final to a csv file
* thats it, its just a csv file

In [6]:
df_big.to_csv(r'/home/anthony/Documents/programing-materals/comp574/DataProject/export_dataframe.csv', index = False, header=True)

#### converts the temp into ferinheit and  dt into a readable date in place of a unix time stamp

In [7]:
df_big.loc[:, "temp"] = df_big["temp"].apply(lambda x: x - 273)
df_big.loc[:, "dt"] = df_big["dt"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%m/%d/%Y'))
df_big

#datetime.datetime.utcfromtimestamp(posix_time).strftime('%Y-%m-%dT%H:%M:%SZ')

,dt,temp,humidity,wind_speed,city,id
0,11/26/2021,5.14,68,1.47,Manchester,1406564545121494235
1,11/26/2021,4.96,71,1.96,Manchester,2121375746036347630
2,11/26/2021,4.50,73,1.80,Manchester,691752794450837760
3,11/26/2021,4.56,71,1.82,Manchester,876220059266076430
4,11/26/2021,5.09,68,0.00,Manchester,207525674555039002
...,...,...,...,...,...,...
19,11/22/2021,9.32,68,3.60,Concord,2121375196280252945
20,11/22/2021,8.32,65,6.17,Concord,1129863098142554144
21,11/22/2021,6.98,63,5.66,Concord,1475739769426937388
22,11/22/2021,6.04,65,6.17,Concord,484227121533424702


## This sectoin handles averaging the columbs

#### Avearge Temp

In [8]:
aveage_temp = df_big.groupby('city')['temp'].mean()
aveage_temp # this needs to be converted to regular temp

city
Concord       2.758462
Manchester    2.936239
keen          0.557692
Name: temp, dtype: float64

### temp variance

In [14]:
aveage_temp_var = df_big.groupby('city')['temp'].var()
aveage_temp_var

city
Concord       17.802837
Manchester    19.556603
keen          12.181340
Name: temp, dtype: float64

#### Average humidity

In [9]:
average_humidity = df_big.groupby('city')['humidity'].mean()
average_humidity

city
Concord       68.786325
Manchester    67.145299
keen          79.982906
Name: humidity, dtype: float64

#### Average wind speed

In [10]:
aveage_wind_speed = df_big.groupby('city')['wind_speed'].mean()
aveage_wind_speed # This is in MPH already, its just so slow that its under 1 mph

city
Concord       3.001624
Manchester    1.411282
keen          3.139744
Name: wind_speed, dtype: float64


#### city mode

In [12]:
city_mode = df_big['city'].mode()
city_mode

0       Concord
1    Manchester
2          keen
dtype: object

In [11]:

# someting to show the temp range in agiven week

# compair average temps via bar graph.

# use the larg graph to do a heat map to see if any of the data is corilated